In [97]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [98]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from math import sqrt, log
sns.set_style("darkgrid")

from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split  

In [99]:
_prints = False

In [100]:
local = True

path = '../data/' 
hotels = pd.read_csv('../data/features_hotels.csv') # à tester, sans ces données

if local : 
    data = pd.read_csv(path+"data.csv")
else :
    url = 'https://drive.google.com/file/d/10sNct4HjylkBudNAQNfvMpI7qgvsspwj/view?usp=sharing'
    path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
    data = pd.read_csv(path)
    
data = data.merge(hotels, on=['hotel_id','city'])
data_test = pd.read_csv(path+'test_set.csv') # ignore index 
data_test['request_number'] = 0
data_test = data_test.merge(hotels, on=['hotel_id','city']) 

for avatar in np.unique(data_test['avatar_id']):
    data_test.loc[data_test['avatar_id'] == avatar, 'request_number'] = data_test['order_requests'].loc[data_test['avatar_id']== avatar] - min(data_test['order_requests'].loc[data_test['avatar_id']== avatar]-1)

data.head()

,city,date,language,mobile,request_number,avatar_id,hotel_id,price,stock,group,brand,parking,pool,children_policy
0,valletta,2,bulgarian,1,1,134,55,109,10,Boss Western,Boss Western,1,0,0
1,valletta,32,bulgarian,1,1,167,55,100,49,Boss Western,Boss Western,1,0,0
2,valletta,32,hungarian,0,1,179,55,100,49,Boss Western,Boss Western,1,0,0
3,valletta,5,danish,1,1,197,55,129,0,Boss Western,Boss Western,1,0,0
4,valletta,36,danish,1,1,203,55,110,64,Boss Western,Boss Western,1,0,0


In [101]:
data.shape

(281393, 14)

In [102]:
data.dtypes

city               object
date                int64
language           object
mobile              int64
request_number      int64
avatar_id           int64
hotel_id            int64
price               int64
stock               int64
group              object
brand              object
parking             int64
pool                int64
children_policy     int64
dtype: object

In [103]:
for col in ['city', 'language', 'avatar_id', 'hotel_id', 'group', 'brand','children_policy'] :
    data[col] = pd.Categorical(data[col],ordered=False)
    data_test[col] = pd.Categorical(data_test[col],ordered=False)
    
data.dtypes

city               category
date                  int64
language           category
mobile                int64
request_number        int64
avatar_id          category
hotel_id           category
price                 int64
stock                 int64
group              category
brand              category
parking               int64
pool                  int64
children_policy    category
dtype: object

In [104]:
colQT = ['request_number', 'stock', 'date']
colQT = ['request_number', 'stock', 'date', 'hotel_id']

In [105]:
if _prints :
    plt.figure()
    sns.histplot(data['price'])
    plt.show()

In [106]:
if _prints :
    plt.figure()
    sns.histplot(data['price'].map(lambda x: x**(1/3)))
    plt.show()

In [107]:
transfo = False
if transfo :
    data['SRprice'] = data['price'].map(lambda x: x**(1/3))

In [108]:
if _prints :
    for col in data.columns:
        plt.figure(figsize=(8,3))
        sns.histplot(data[col])
        plt.xticks(rotation=45)
        plt.show()

In [109]:
if _prints :
    for col in data_test.columns:
        plt.figure(figsize=(8,3))
        sns.histplot(data_test[col])
        plt.xticks(rotation=45)
        plt.show()

In [27]:
dataDum = pd.get_dummies(data[['city','language','mobile','group','brand','parking','pool','children_policy']])
del dataDum['city_amsterdam']
del dataDum['language_hungarian']
# del dataDum['hotel_id_0']
del dataDum['group_Accar Hotels']
del dataDum['brand_8 Premium']
del dataDum['children_policy_0']

# on Test Set

dataDum_test = pd.get_dummies(data_test[['city','language','mobile','group','brand','parking','pool','children_policy']])
del dataDum_test['city_amsterdam']
del dataDum_test['language_hungarian']
# del dataDum_test['hotel_id_0']
del dataDum_test['group_Accar Hotels']
del dataDum_test['brand_8 Premium']
del dataDum_test['children_policy_0']

In [110]:
dataDum = pd.get_dummies(data[['city','language','mobile']])
del dataDum['city_amsterdam']
del dataDum['language_hungarian']

# on Test Set

dataDum_test = pd.get_dummies(data_test[['city','language','mobile']])
del dataDum_test['city_amsterdam']
del dataDum_test['language_hungarian']

In [111]:
# First work without order request 
# del colQT[0]

In [112]:
dataQt = data[colQT]
dataQt_test = data_test[colQT]

In [113]:
df = pd.concat([dataDum,dataQt],axis=1)
df_test = pd.concat([dataDum_test,dataQt_test],axis=1)
df.head()

,mobile,city_copenhagen,city_madrid,city_paris,city_rome,city_sofia,city_valletta,city_vienna,city_vilnius,language_austrian,...,language_portuguese,language_romanian,language_slovakian,language_slovene,language_spanish,language_swedish,request_number,stock,date,hotel_id
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,10,2,55
1,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,49,32,55
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,49,32,55
3,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,5,55
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,64,36,55


In [114]:
# variable à expliquer réelle
if transfo :
    Yr = data['SRprice']
else :
    Yr = data['price']

In [56]:
# séparation pour les tests et tunner les modèles
X_train,X_test,Yr_train,Yr_test=train_test_split(df,Yr,test_size=.25,random_state=11) #25% pour le test_size
submit = False

In [115]:
# séparation pour submit
X_train = df
X_test = df_test
Yr_train = Yr
Yr_test = None
submit = True

In [116]:
# standardisation des données
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(X_train)  
Xr_train = scaler.transform(X_train)  
# Meme transformation sur le test
Xr_test = scaler.transform(X_test)

# Boosting

In [88]:
# définition des paramètres
boost = GradientBoostingRegressor()
# apprentissage
rfFit = boost.fit(Xr_train,Yr_train)

In [89]:
if not submit:
# erreur de prédiction sur le test
    1-rfFit.score(Xr_test,Yr_test)

In [90]:
# prédiction
y_pred = rfFit.predict(Xr_test)
if not submit:
    print("MSE =",mean_squared_error(y_pred,Yr_test))
    print("R2 =",r2_score(Yr_test,y_pred))

# Random Forest

In [ ]:
# définition des paramètres
forest = RandomForestRegressor(n_estimators=500,
                               max_depth=None,
                               min_samples_split=2, min_samples_leaf=1, 
                               max_features='auto', max_leaf_nodes=None,
                               bootstrap=True, oob_score=True)
# apprentissage
rfFit = forest.fit(Xr_train,Yr_train)
print(1-rfFit.oob_score_)

In [ ]:
param=[{"max_features":list(range(2,10,1))}]
rf= GridSearchCV(RandomForestRegressor(n_estimators=100),param,cv=5,n_jobs=-1)
rfOpt=rf.fit(Xr_train, Yr_train)
# paramètre optimal
print("Meilleur score = %f, Meilleur paramètre = %s" % (1. - rfOpt.best_score_,rfOpt.best_params_))

In [ ]:
if not submit:
# erreur de prévision sur le test
    1-rfOpt.score(Xr_test,Yr_test)

In [ ]:
# prévision
y_pred = rfOpt.predict(Xr_test)
if not submit :
    print("MSE=",mean_squared_error(y_pred,Yr_test))
    print("R2=",r2_score(Yr_test,y_pred))

# Export predictions

In [ ]:
y_pred

In [ ]:
# save
if transfo:
    y_pred = y_pred**3
if submit :
    sub = pd.DataFrame(y_pred)
    sub.to_csv('sub_randomF_hotelid.csv',index=True, header=['price'], index_label = 'index')

In [ ]:
sns.histplot(y_pred)

In [37]:
data_test['price'] = y_pred

In [38]:
data_test

,index,order_requests,city,date,language,mobile,avatar_id,hotel_id,stock,request_number,group,brand,parking,pool,children_policy,price
0,0,1,vilnius,21,romanian,0,1,161,46,1,Boss Western,J.Halliday Inn,1,0,0,112.317450
1,9,2,vilnius,18,romanian,0,1,161,36,2,Boss Western,J.Halliday Inn,1,0,0,111.623937
2,254,30,vilnius,20,romanian,1,12,161,44,1,Boss Western,J.Halliday Inn,1,0,0,111.493849
3,263,31,vilnius,18,romanian,1,12,161,36,2,Boss Western,J.Halliday Inn,1,0,0,112.682690
4,272,32,vilnius,16,bulgarian,0,12,161,30,3,Boss Western,J.Halliday Inn,1,0,0,113.653206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6639,5187,662,rome,39,slovene,1,771,856,39,1,Accar Hotels,Ibas,1,0,0,71.922692
6640,5426,693,rome,34,slovakian,1,514,856,26,1,Accar Hotels,Ibas,1,0,0,70.747590
6641,5920,753,rome,36,italian,0,768,856,31,1,Accar Hotels,Ibas,1,0,0,71.997737
6642,6350,811,rome,34,slovene,1,766,856,26,1,Accar Hotels,Ibas,1,0,0,72.039572


In [44]:
data.loc[data['avatar_id'] == 171]

,city,date,language,mobile,request_number,avatar_id,hotel_id,price,stock,group,brand,parking,pool,children_policy,SRprice
271645,sofia,17,greek,0,1,171,683,89,22,Chillton Worldwide,Tripletree,0,0,0,4.464745
271698,sofia,15,greek,0,2,171,683,89,16,Chillton Worldwide,Tripletree,0,0,0,4.464745
271717,sofia,14,greek,0,3,171,683,92,13,Chillton Worldwide,Tripletree,0,0,0,4.514357
271750,sofia,6,greek,0,4,171,683,90,0,Chillton Worldwide,Tripletree,0,0,0,4.481405
271768,sofia,17,greek,0,1,171,470,75,14,Independant,Independant,0,0,0,4.217163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281076,sofia,6,greek,0,4,171,871,89,0,Yin Yang,Ardisson,0,0,0,4.464745
281164,sofia,14,greek,0,3,171,899,104,14,Morriott International,CourtYord,0,0,0,4.702669
281194,sofia,6,greek,0,4,171,899,102,6,Morriott International,CourtYord,0,0,0,4.672329
281282,sofia,14,greek,0,3,171,262,104,10,Boss Western,J.Halliday Inn,1,0,0,4.702669


In [ ]:
592 788 171 239